In [5]:
import sys
path_to_challenge_sncf_2024 = '/Users/gurvanrichardeau/Python_projects/LocalCodes/challenge_sncf_2024'
sys.path.append(path_to_challenge_sncf_2024 + '/utils')

from PandasToolsFunction import *
from DataPreprocessingTools import *
import pandas as pd
from tqdm import tqdm
import json

In [75]:
class AverageModel():
    def __init__(self, train_data: pd.DataFrame, day_types = ['job', 'ferie', 'vacances']) -> None:
        self.train_data = train_data
        self.day_types = day_types
        self.station_day_type_map = {}

        self.submission = None

    def basic_train(self):
        """
        Computes day_type and station wise averages.
        """
        data = self.train_data.copy() # Ensure it does not modify original data.

        for station in tqdm(data['station'].unique()):
            station_count = 0
            for day_type in self.day_types: # Our day types will form a partition.
                mask = (data['station'] == station) & (data['day_type'] == day_type)
                station_day_type_data = data[mask]
                avg = station_day_type_data['y'].mean()
                key = f"{station}_{day_type}"  
                self.station_day_type_map[key] = avg

    def train(self, mode='basic'):
        self.station_day_type_map = {} # Reset the model.
        if mode == 'basic':
            self.basic_train()
        else :
            pass
 
    def save(self, name = "avg_model"):
        with open("avg_saved_models/" + name + ".json", 'w') as json_file:
            json.dump(self.station_day_type_map, json_file)

    def load(self, name="avg_model"):
        with open("avg_saved_models/" + name + ".json", 'r') as json_file:
            self.station_day_type_map = json.load(json_file)

    def infer(self, test_data_init: pd.DataFrame):
        """
        Assume test_data has a column station and one for each day type.
        """
        test_data = test_data_init.copy()
        

        for index, row in test_data.iterrows():
            station, day_type = row['station'], row['day_type']
            key = f"{station}_{day_type}"
            test_data.at[index, 'y'] = self.station_day_type_map[key]

        self.submission = test_data[['index', 'y']]
        return test_data
    
    def save_submission(self, name = "submission"):
        self.submission.to_csv("submissions/" + name + ".csv", index=False)    

In [80]:
# Charger les données d'entraînement
train_data = pd.read_csv('train_data_woCov_with_day_of_week.csv')
#y_train_data = pd.read_csv('y_train_sncf.csv')

#train_data = trainf_data
#train_data['y'] = y_train_data['y']
train_data = data_day_typer(train_data)

test_data = pd.read_csv('test_data_day_type1.csv')
#test_data = data_day_typer(test_data)

In [81]:
avg_model = AverageModel(train_data)

In [82]:
avg_model.train('basic')

100%|██████████| 439/439 [01:15<00:00,  5.81it/s]


In [83]:
avg_model.save('woCovid_basic_avg_model')

In [61]:
avg_model.load()

In [84]:
result = avg_model.infer(test_data)
avg_model.save_submission('submission_basic_avg_woCovid')